# Cost for gate infidelity and parameter for Adam optimizer. And setup both for manual and Auto-grad are same

In [4]:

from qoc import grape_schroedinger_discrete
from qoc.standard import (Adam,TargetStateInfidelity,TargetStateInfidelityTime,
                          conjugate_transpose,matrix_to_column_vector_list,
                          get_annihilation_operator,
                          get_creation_operator,
                          SIGMA_Z,
                          generate_save_file_path,)
from qoc.models.operationpolicy import OperationPolicy
from qoc.models import (Dummy, EvolveSchroedingerDiscreteState,
                        EvolveSchroedingerResult,
                        GrapeSchroedingerDiscreteState,
                        GrapeSchroedingerResult,
                        InterpolationPolicy,
                        MagnusPolicy,
                        ProgramType,)
import numpy as np

# Define the system.
HILBERT_SIZE = 2
ANNIHILATION_OPERATOR = get_annihilation_operator(HILBERT_SIZE)
CREATION_OPERATOR = get_creation_operator(HILBERT_SIZE)
# E.q. 19 (p. 6) of https://arxiv.org/abs/1904.06560.
sigmax=ANNIHILATION_OPERATOR +CREATION_OPERATOR
sigmay=-1j*ANNIHILATION_OPERATOR+1j*CREATION_OPERATOR
# E.q. 19 (p. 6) of https://arxiv.org/abs/1904.06560.
H_SYSTEM_0 = SIGMA_Z / 2
# Only real control amplitutdes are supported!
hamiltonian = lambda controls, time: (H_SYSTEM_0
                                      + controls[0] * sigmax
                                      + controls[1]*sigmay)


# Define the optimization.
COMPLEX_CONTROLS = False
CONTROL_COUNT = 2
EVOLUTION_TIME = 10 # nanoseconds
CONTROL_EVAL_COUNT = SYSTEM_EVAL_COUNT = EVOLUTION_TIME + 1
ITERATION_COUNT = 1000

# Define output.
LOG_ITERATION_STEP = 1
SAVE_ITERATION_STEP = 1
SAVE_PATH = "./out"
SAVE_FILE_NAME = "transmon_pi"
SAVE_FILE_PATH = generate_save_file_path(SAVE_FILE_NAME, SAVE_PATH)
INITIAL_STATES = matrix_to_column_vector_list(np.eye(2))
# we could have equivalently done
# initial_state0 = np.array([[1], [0]])
# initial_state1 = np.array([[0], [1]])
# initial_states = np.stack((initial_state0, initial_state1))
target_unitary = np.array([[0, 1], [1, 0]])
target_states = matrix_to_column_vector_list(target_unitary)
# we could have equivalently done
# target_state0 = np.array([[0], [1]])
# target_state1 = np.array([[1], [0]])
# target_states = np.stack((target_state0, target_state1))


#cost_multiplier is hyperprameter of each cost function
#Set neglect_relative_phase as True, relative phase of each column of gate U will be ignored.
#cost_eval_step :: int >= 1- This value determines how often step-costs are evaluated.The units of this value are in system_eval steps. E.g. if this value is 2,step-costs will be computed every 2 system_eval steps.

cost_1=TargetStateInfidelityTime(system_eval_count=SYSTEM_EVAL_COUNT,
                                                     target_states=target_states,cost_multiplier=1,
                                                     neglect_relative_phase=False,cost_eval_step=1)
cost_2=TargetStateInfidelity(target_states=target_states,cost_multiplier=1,neglect_relative_phase=False)
COSTS = [cost_1]
CONTROL_HAMILTONIAN=[sigmax,sigmay]
manual_parameter={"control_hamiltonian":CONTROL_HAMILTONIAN,"manual_gradient_mode":False,"tol":1e-16}


optimizer=Adam(beta_1=0.9, beta_2=0.999, clip_grads=None,
                 epsilon=1e-8, learning_rate=1e-3,
                 learning_rate_decay=None, operation_policy=OperationPolicy.CPU,
                 scale_grads=None)
# a class to define the Adam optimizer
#     This implementation follows the original algorithm
#     https://arxiv.org/abs/1412.6980.
#     Fields:
#     apply_clip_grads :: bool - see clip_grads
#     apply_learning_rate_decay :: bool - see learning_rate_decay
#     apply_scale_grads :: bool - see scale_grads
#     beta_1 :: float - gradient decay bias
#     beta_2 :: float - gradient squared decay bias
#     clip_grads :: float - the maximum absolute value at which the gradients
#         should be element-wise clipped, if not set, the gradients will
#         not be clipped
#     epsilon :: float - fuzz factor
#     gradient_moment :: numpy.ndarray - running optimization variable
#     gradient_square_moment :: numpy.ndarray - running optimization variable
#     initial_learning_rate :: float - the initial step size
#     iteration_count :: int - the current count of iterations performed
#     learning_rate :: float - the current step size
#     learning_rate_decay :: float - the number of iterations it takes for
#         the learning rate to decay by 1/e, if not set, no decay is
#         applied
#     operation_policy
#     name :: str - identifier for the optimizer
#     scale_grads :: float - the value to scale the norm of the gradients to,
#         if not set, the gradients will not be scaled
def main():
    result = grape_schroedinger_discrete(CONTROL_COUNT, CONTROL_EVAL_COUNT,
                                         COSTS, EVOLUTION_TIME, hamiltonian,
                                         INITIAL_STATES, SYSTEM_EVAL_COUNT,
                                         complex_controls=False,
                                    cost_eval_step=1,
                                    impose_control_conditions=None,
                                    initial_controls=None,
                                    interpolation_policy=InterpolationPolicy.LINEAR,
                                    iteration_count=1000,
                                    log_iteration_step=10,
                                    magnus_policy=MagnusPolicy.M2,
                                    max_control_norms=None,
                                    min_error=0,
                                    optimizer=optimizer,
                                    save_file_path=None,
                                    save_intermediate_states=False,
                                    save_iteration_step=0,manual_parameter=manual_parameter)
#
#Args:
#    control_count :: int - This is the number of control parameters that qoc should
#        optimize over. I.e. it is the length of the `controls` array passed
#        to the hamiltonian.
#    control_eval_count :: int >= 2 - This value determines where definite values
#        of the control parameters are evaluated. This value is used as:
#        `control_eval_times`= numpy.linspace(0, `evolution_time`, `control_eval_count`).
#   costs :: iterable(qoc.models.cost.Cost) - This list specifies all
#       the cost functions that the optimizer should evaluate. This list
#        defines the criteria for an "optimal" control set.
#    evolution_time :: float - This value specifies the duration of the
#        system's evolution.
#    hamiltonian :: (controls :: ndarray (control_count), time :: float)
#                   -> hamiltonian_matrix :: ndarray (hilbert_size x hilbert_size)
#        - This function provides the system's hamiltonian given a set
#        of control parameters and a time value.
#    initial_states :: ndarray (state_count x hilbert_size x 1)
#        - This array specifies the states that should be evolved under the
#        specified system. These are the states at the beginning of the evolution.
#    system_eval_count :: int >= 2 - This value determines how many times
#        during the evolution the system is evaluated, including the
#        initial value of the system. For the schroedinger evolution,
#        this value determines the time step of integration.
#        This value is used as:
#        `system_eval_times` = numpy.linspace(0, `evolution_time`, `system_eval_count`).
#
#    complex_controls :: bool - This value determines if the control parameters
#        are complex-valued. If some controls are real only or imaginary only
#        while others are complex, real only and imaginary only controls
#        can be simulated by taking the real or imaginary part of a complex control.
#    cost_eval_step :: int >= 1- This value determines how often step-costs are evaluated.
#         The units of this value are in system_eval steps. E.g. if this value is 2,
#         step-costs will be computed every 2 system_eval steps.
#    impose_control_conditions :: (controls :: (control_eval_count x control_count))
#                                -> (controls :: (control_eval_count x control_count))
#        - This function is called after every optimization update. Example uses
#        include setting boundary conditions on the control parameters.                             
#    initial_controls :: ndarray (control_step_count x control_count)
#       - This array specifies the control parameters at each
#        control step. These values will be used to determine the `controls`
#        argument passed to the `hamiltonian` function at each time step for
#        the first iteration of optimization.
#    interpolation_policy :: qoc.models.interpolationpolicy.InterpolationPolicy
#        - This value specifies how control parameters should be
#        interpreted at points where they are not defined.
#    iteration_count :: int - This value determines how many total system
#        evolutions the optimizer will perform to determine the
#       optimal control set.
#    log_iteration_step :: int - This value determines how often qoc logs
#        progress to stdout. This value is specified in units of system steps,
#        of which there are `control_step_count` * `system_step_multiplier`.
#        Set this value to 0 to disable logging.
#    magnus_policy :: qoc.models.magnuspolicy.MagnusPolicy - This value
#        specifies what method should be used to perform the magnus expansion
#        of the system matrix for ode integration. Choosing a higher order
#        magnus expansion will yield more accuracy, but it will
#        result in a longer compute time.
#    max_control_norms :: ndarray (control_count) - This array
#       specifies the element-wise maximum norm that each control is
#        allowed to achieve. If, in optimization, the value of a control
#        exceeds its maximum norm, the control will be rescaled to
#        its maximum norm. Note that for non-complex values, this
#        feature acts exactly as absolute value clipping.
#    min_error :: float - This value is the threshold below which
#        optimization will terminate.
#    optimizer :: class instance - This optimizer object defines the
#        gradient-based procedure for minimizing the total contribution
#        of all cost functions with respect to the control parameters.
#    save_file_path :: str - This is the full path to the file where
#        information about program execution will be stored.
#        E.g. "./out/foo.h5"
#    save_intermediate_densities :: bool - If this value is set to True,
#        qoc will write the densities to the save file after every
#        system_eval step.
#    save_intermediate_states :: bool - If this value is set to True,
#        qoc will write the states to the save file after every
#        system_eval step.
#    save_iteration_step :: int - This value determines how often qoc
#        saves progress to the save file specified by `save_file_path`.
#        This value is specified in units of system steps, of which
#       there are `control_step_count` * `system_step_multiplier`.
#        Set this value to 0 to disable saving.
        
if __name__ == "__main__":
    main()




iter   |   total error  |    grads_l2   
  0    | 9.78697344e-01 | 3.53190944e-01
  10   | 9.62152856e-01 | 4.77545945e-01
  20   | 9.39886294e-01 | 5.91627441e-01
  30   | 9.11757318e-01 | 6.90381060e-01
  40   | 8.78377852e-01 | 7.66530000e-01
  50   | 8.41231421e-01 | 8.13572513e-01
  60   | 8.02463194e-01 | 8.28174726e-01
  70   | 7.64448944e-01 | 8.11437601e-01
  80   | 7.29302665e-01 | 7.68743084e-01
  90   | 6.98487899e-01 | 7.08377158e-01
 100   | 6.72647648e-01 | 6.39517178e-01
 110   | 6.51673708e-01 | 5.70321921e-01
 120   | 6.34941983e-01 | 5.06695123e-01
 130   | 6.21594882e-01 | 4.51916448e-01
 140   | 6.10771059e-01 | 4.06976741e-01
 150   | 6.01739113e-01 | 3.71281227e-01
 160   | 5.93943442e-01 | 3.43399960e-01
 170   | 5.86994430e-01 | 3.21660991e-01


KeyboardInterrupt: 

# Cost for forbidden states

In [5]:


import numpy as np

from qoc.core import grape_schroedinger_discrete
from qoc.standard import (ForbidStates, SIGMA_X, SIGMA_Y,)
hilbert_size = 4
hamiltonian_matrix = np.divide(1, 2) * (np.kron(SIGMA_X, SIGMA_X)
                                            + np.kron(SIGMA_Y, SIGMA_Y))
hamiltonian = lambda controls, t: (controls[0] * hamiltonian_matrix)
initial_states = np.array([[[0], [1], [0], [0]]])
forbidden_states = np.array([[[[0], [1], [0], [0]]]])
control_count = 1
evolution_time = 10
control_eval_count = system_eval_count = 11
costs = [ForbidStates(forbidden_states=forbidden_states, system_eval_count=system_eval_count,cost_eval_step=1,
                 cost_multiplier=1.,)]
iteration_count = 1000
log_iteration_step = 1
manual_parameter={"control_hamiltonian":[hamiltonian_matrix],"manual_gradient_mode":False,"tol":1e-16}
def main():
    result = grape_schroedinger_discrete(control_count, control_eval_count,
                                         costs, evolution_time,
                                         hamiltonian, initial_states,
                                         system_eval_count,
                                         iteration_count=iteration_count,
                                         log_iteration_step=log_iteration_step,
                                         manual_parameter=manual_parameter
                                         )
if __name__ == "__main__":
    main()

iter   |   total error  |    grads_l2   
  0    | 6.91162432e-01 | 1.62820647e+00
  1    | 6.86353720e-01 | 1.62888413e+00
  2    | 6.81546931e-01 | 1.62920640e+00
  3    | 6.76743260e-01 | 1.62917491e+00
  4    | 6.71943924e-01 | 1.62879144e+00
  5    | 6.67150164e-01 | 1.62805793e+00
  6    | 6.62363242e-01 | 1.62697649e+00
  7    | 6.57584445e-01 | 1.62554942e+00
  8    | 6.52815075e-01 | 1.62377922e+00
  9    | 6.48056455e-01 | 1.62166858e+00
  10   | 6.43309924e-01 | 1.61922041e+00
  11   | 6.38576836e-01 | 1.61643784e+00
  12   | 6.33858555e-01 | 1.61332423e+00
  13   | 6.29156457e-01 | 1.60988315e+00
  14   | 6.24471923e-01 | 1.60611842e+00
  15   | 6.19806342e-01 | 1.60203412e+00
  16   | 6.15161102e-01 | 1.59763455e+00
  17   | 6.10537593e-01 | 1.59292426e+00
  18   | 6.05937198e-01 | 1.58790806e+00
  19   | 6.01361297e-01 | 1.58259100e+00
  20   | 5.96811258e-01 | 1.57697839e+00
  21   | 5.92288438e-01 | 1.57107576e+00
  22   | 5.87794179e-01 | 1.56488891e+00
  23   | 5.83329

KeyboardInterrupt: 

# Cost for controls

In [12]:
"""

"""



from qoc import grape_schroedinger_discrete
from qoc.standard import (TargetStateInfidelity,
                          conjugate_transpose,
                          get_annihilation_operator,
                          get_creation_operator,
                          SIGMA_Z,
                          generate_save_file_path, ForbidStates, ControlArea, ControlNorm, ControlVariation,
                          ControlBandwidthMax,)
import numpy as np

# Define the system.
HILBERT_SIZE = 2
ANNIHILATION_OPERATOR = get_annihilation_operator(HILBERT_SIZE)
CREATION_OPERATOR = get_creation_operator(HILBERT_SIZE)
sigmax=ANNIHILATION_OPERATOR +CREATION_OPERATOR
sigmay=-1j*ANNIHILATION_OPERATOR+1j*CREATION_OPERATOR
# E.q. 19 (p. 6) of https://arxiv.org/abs/1904.06560.
H_SYSTEM_0 = SIGMA_Z / 2
# Use a + a^{dagger} as the drive term to control.
hamiltonian = lambda controls, time: (H_SYSTEM_0
                                      + controls[0] * sigmax
                                      + controls[1]*sigmay)
CONTROL_HAMILTONIAN=[sigmax,sigmay]
# Define the problem.
INITIAL_STATE_0 = np.array([[1], [0]])
TARGET_STATE_0 = np.array([[0], [1]])
INITIAL_STATES = np.stack((INITIAL_STATE_0,), axis=0)
TARGET_STATES = np.stack((TARGET_STATE_0,), axis=0)
FORBIDDEN_STATES = np.stack((INITIAL_STATES,), axis=0)

# Define the optimization.
COMPLEX_CONTROLS = False
CONTROL_COUNT = 2
EVOLUTION_TIME = 10 # nanoseconds
CONTROL_EVAL_COUNT  = 5
SYSTEM_EVAL_COUNT=5
ITERATION_COUNT = 1000
max_norm = 2
initial_controls = np.array([[0.1,0.1],[0.2,0.2],[0.3,0.3],[0.4,0.4],[0.5,0.5]])

# Define output.
LOG_ITERATION_STEP = 1
SAVE_ITERATION_STEP = 1
SAVE_PATH = "./out"
SAVE_FILE_NAME = "transmon_pi"
SAVE_FILE_PATH = generate_save_file_path(SAVE_FILE_NAME, SAVE_PATH)
manual_parameter={"control_hamiltonian":CONTROL_HAMILTONIAN,"manual_gradient_mode":False,"tol":1e-16}


# This cost penalizes control frequencies above a set maximum.

#     Fields:
#     max_bandwidths :: ndarray (CONTROL_COUNT) - This array contains the maximum allowed bandwidth of each control.
#     control_count
#     freqs :: ndarray (CONTROL_EVAL_COUNT) - This array contains the frequencies of each of the controls.
#     name
#     requires_step_evaluation
MAX_BANDWIDTH_0 = 0.001 # GHz
MAX_BANDWIDTHS = np.array((MAX_BANDWIDTH_0,MAX_BANDWIDTH_0))
cost1=ControlBandwidthMax(CONTROL_COUNT, CONTROL_EVAL_COUNT,
                                 EVOLUTION_TIME, MAX_BANDWIDTHS)

# This cost penalizes the area under the
#     function of time generated by the discrete control parameters.
max_control_norms=0.001*np.ones(2)
cost2=ControlArea(control_count=CONTROL_COUNT,
                 control_eval_count=CONTROL_EVAL_COUNT,
                 cost_multiplier=1.,
                 max_control_norms=max_control_norms,)



# This cost penalizes the value of the norm of the control parameters.

#     Fields:
#     control_weights :: ndarray (control_eval_count x control_count)
#         - These weights, each of which should be no greater than 1,
#         represent the factor by which each control's magnitude is penalized.
#         If no weights are specified, each control's magnitude is penalized
#         equally.
cost3=ControlNorm(control_count=CONTROL_COUNT,
                 control_eval_count=CONTROL_EVAL_COUNT,
                 control_weights=None,
                 cost_multiplier=1.,
                 max_control_norms=None)
# This cost penalizes the variations of the control parameters
#     from one `control_eval_step` to the next.
# manual only supports first and second order so far.
cost4=ControlVariation(control_count=CONTROL_COUNT,
                 control_eval_count=CONTROL_EVAL_COUNT,
                 cost_multiplier=1.,
                 max_control_norms=None,
                 order=1)

COSTS = [cost4]
def main():
    result = grape_schroedinger_discrete(CONTROL_COUNT, CONTROL_EVAL_COUNT,
                                         COSTS, EVOLUTION_TIME, hamiltonian,
                                         INITIAL_STATES, SYSTEM_EVAL_COUNT,
                                         initial_controls=initial_controls,
                                         complex_controls=COMPLEX_CONTROLS,
                                         iteration_count=ITERATION_COUNT,
                                         log_iteration_step=LOG_ITERATION_STEP,
                                         save_file_path=SAVE_FILE_PATH,
                                         save_iteration_step=SAVE_ITERATION_STEP,
                                         manual_parameter=manual_parameter
                                         )


if __name__ == "__main__":
    main()


QOC is saving this optimization run to ./out/00020_transmon_pi.h5.
iter   |   total error  |    grads_l2   
  0    | 5.00000000e-03 | 2.50000000e-02
  1    | 4.95025004e-03 | 2.47512628e-02
  2    | 4.90054639e-03 | 2.46864136e-02
  3    | 4.85116549e-03 | 2.46370142e-02
  4    | 4.80214774e-03 | 2.45393103e-02
  5    | 4.75324461e-03 | 2.43730158e-02
  6    | 4.70476451e-03 | 2.41946196e-02
  7    | 4.65648703e-03 | 2.40274410e-02
  8    | 4.60850766e-03 | 2.38971192e-02
  9    | 4.56079252e-03 | 2.37974837e-02
  10   | 4.51333122e-03 | 2.37105796e-02
  11   | 4.46619516e-03 | 2.36241738e-02
  12   | 4.41938554e-03 | 2.35231040e-02
  13   | 4.37283930e-03 | 2.33960365e-02
  14   | 4.32659018e-03 | 2.32493559e-02
  15   | 4.28067163e-03 | 2.30967192e-02
  16   | 4.23505554e-03 | 2.29498159e-02
  17   | 4.18973989e-03 | 2.28161688e-02
  18   | 4.14473721e-03 | 2.26985258e-02
  19   | 4.10004013e-03 | 2.25930455e-02
  20   | 4.05565371e-03 | 2.24921160e-02
  21   | 4.01159711e-03 | 2.238

 199   | 3.13409873e-04 | 6.25386678e-03
 200   | 3.07805290e-04 | 6.19769636e-03
 201   | 3.02287164e-04 | 6.14188969e-03
 202   | 2.96854429e-04 | 6.08644870e-03
 203   | 2.91506028e-04 | 6.03137320e-03
 204   | 2.86240913e-04 | 5.97666014e-03
 205   | 2.81058042e-04 | 5.92230601e-03
 206   | 2.75956384e-04 | 5.86830960e-03
 207   | 2.70934914e-04 | 5.81467205e-03
 208   | 2.65992619e-04 | 5.76139440e-03
 209   | 2.61128492e-04 | 5.70847563e-03
 210   | 2.56341534e-04 | 5.65591299e-03
 211   | 2.51630755e-04 | 5.60370391e-03
 212   | 2.46995176e-04 | 5.55184745e-03
 213   | 2.42433823e-04 | 5.50034421e-03
 214   | 2.37945731e-04 | 5.44919456e-03
 215   | 2.33529946e-04 | 5.39839716e-03
 216   | 2.29185520e-04 | 5.34794941e-03
 217   | 2.24911514e-04 | 5.29784922e-03
 218   | 2.20706997e-04 | 5.24809605e-03
 219   | 2.16571048e-04 | 5.19869016e-03
 220   | 2.12502754e-04 | 5.14963114e-03
 221   | 2.08501208e-04 | 5.10091735e-03
 222   | 2.04565514e-04 | 5.05254657e-03
 223   | 2.00694

 414   | 2.17171979e-06 | 5.20593857e-04
 415   | 2.11097470e-06 | 5.13261466e-04
 416   | 2.05182762e-06 | 5.06019883e-04
 417   | 1.99423949e-06 | 4.98868178e-04
 418   | 1.93817213e-06 | 4.91805429e-04
 419   | 1.88358821e-06 | 4.84830719e-04
 420   | 1.83045125e-06 | 4.77943138e-04
 421   | 1.77872556e-06 | 4.71141783e-04
 422   | 1.72837629e-06 | 4.64425757e-04
 423   | 1.67936935e-06 | 4.57794171e-04
 424   | 1.63167142e-06 | 4.51246141e-04
 425   | 1.58524995e-06 | 4.44780791e-04
 426   | 1.54007312e-06 | 4.38397249e-04
 427   | 1.49610985e-06 | 4.32094652e-04
 428   | 1.45332976e-06 | 4.25872143e-04
 429   | 1.41170316e-06 | 4.19728871e-04
 430   | 1.37120106e-06 | 4.13663992e-04
 431   | 1.33179511e-06 | 4.07676669e-04
 432   | 1.29345765e-06 | 4.01766071e-04
 433   | 1.25616164e-06 | 3.95931371e-04
 434   | 1.21988068e-06 | 3.90171754e-04
 435   | 1.18458895e-06 | 3.84486406e-04
 436   | 1.15026129e-06 | 3.78874522e-04
 437   | 1.11687307e-06 | 3.73335304e-04
 438   | 1.08440

 616   | 2.53380490e-09 | 1.77871523e-05
 617   | 2.43832173e-09 | 1.74739699e-05
 618   | 2.34759150e-09 | 1.71726660e-05
 619   | 2.26365671e-09 | 1.70086816e-05
 620   | 2.19246208e-09 | 1.70277639e-05
 621   | 2.15098796e-09 | 1.78307218e-05
 622   | 2.18761090e-09 | 2.01288534e-05
 623   | 2.44019919e-09 | 2.62355291e-05
 624   | 3.29951353e-09 | 3.84641697e-05
 625   | 5.84342998e-09 | 6.13351942e-05
 626   | 1.27802819e-08 | 9.91124872e-05
 627   | 2.90531755e-08 | 1.55344541e-04
 628   | 5.46941166e-08 | 2.16107803e-04
 629   | 6.26202039e-08 | 2.32690239e-04
 630   | 2.62107931e-08 | 1.48645578e-04
 631   | 1.59168093e-09 | 1.83096646e-05
 632   | 2.65839850e-08 | 1.50032912e-04
 633   | 3.44071484e-08 | 1.71654377e-04
 634   | 5.84049172e-09 | 6.52491563e-05
 635   | 9.05793200e-09 | 8.45740743e-05
 636   | 2.58024529e-08 | 1.48300965e-04
 637   | 8.39464010e-09 | 8.13201860e-05
 638   | 3.98749503e-09 | 5.21472464e-05
 639   | 1.82228303e-08 | 1.24221834e-04
 640   | 7.49306

KeyboardInterrupt: 